# Tower of Hanoi Example

This [source](https://en.wikipedia.org/wiki/Tower_of_Hanoi) describes the puzzle called the **Tower of Hanoi** which is the focus of this value iteration example. In this puzzle, we have a number of pegs and a number of disks with different sizes. The puzzle starts with all disks on the leftmost peg in an order so that the bottom disk is the biggest and the disk sizes decrease until the top disk which is the smallest. The puzzle is considered solved when all disks are on the rightmost peg in order from biggest to smallest (bottom to top). Each action consists of moving a single disk from one peg to another. An action is only valid if the order of the disks is preserved so that a bigger disk cannot be placed on a smaller disk. 

We can model this puzzle as a Markov Decision Process (MDP) and then solve the optimal order of actions using value iteration! 

We can think of a state as the order of disks on each peg. The disks in order of biggest to largest for 4 disks is `[4, 3, 2, 1]`. An example state is `((1,), (2,), (4, 3))`. This means that on peg 0 (leftmost peg), there is the smallest disk - disk 1. On peg 1 (middle peg), there is disk 2. On peg 2 (rightmost peg), disk 4 is at the bottom and disk 3 is on top of it. An action is moving one disk from one peg to another. For example:
- State: `((1,), (2,), (4, 3))`
- Action: `'0->1'`
- Next state: `((), (2, 1), (4, 3))`

We need to add rewards to the system - we will state that on completing the puzzle you get a reward of 1.

## Getting Started

To begin, we need to install the package `value_iteration` to use its functions.

In [1]:
%%capture
# Install required packages
! pip install git+https://github.com/CassandraDurr/value_iteration.git

Next, import the relevant packages for this example.

In [2]:
from itertools import product

from value_iteration import ValueIteration, MDP

## Setting up the MDP for the Tower of Hanoi
Next, there is some work to be done in setting up the puzzle as an MDP. We first need a function that helps us set up all the states in the state space depending on the choice of the number of disks and the number of pegs. The function `generate_all_states` creates the state space.

In [3]:
def generate_all_states(num_disks: int = 4, num_pegs: int = 3) -> list[list[int]]:
    """Generate all valid Tower of Hanoi states with a given number of pegs and disks.
    Each state is a list of pegs and each peg is a list of disks.
    The list of disks are in strictly decreasing order (largest on bottom).

    Args:
        num_disks (int, optional): The number of disks. Defaults to 4.
        num_pegs (int, optional): The number of pegs. Defaults to 3.

    Returns:
        list[list[int]]: list of states where a state is a list of disk-peg assignments.
    """
    # Store states
    states = []

    # Get all assignments with form like: (0, 1, 2, 2)
    # This means that disk 1 is on peg 0, disk 2 is on peg 1, disk 3 and 4 are on peg 2.
    # Disk 1 is the smallest, disk 2 next smallest, ...
    # Mathematically: assignment[i] = the peg where disk (i+1) should go

    for assignment in product(range(num_pegs), repeat=num_disks):
        # Use assignment to build (disk_number, assigned_peg) list in descending disk order
        # We use descending order so it's easier to adhere to the Tower of Hanoi rules
        # (biggest to smallest disk order)
        disk_peg_pairs = [
            (disk, assignment[disk - 1])
            for disk in range(num_disks, 0, -1)  # For 4 disks -> 4, 3, 2, 1
        ]
        # For assignment (0, 1, 2, 2): [(4, 2), (3, 2), (2, 1), (1, 0)]
        # Disk 4 is on peg 2, disk 3 in on peg 2, disk 2 is on peg 1 and disk 1 is on peg 0

        # Build empty pegs storage to add disks to
        pegs = [[] for _ in range(num_pegs)]

        # Now place pegs using the disk-peg pairs
        for disk, peg_index in disk_peg_pairs:
            pegs[peg_index].append(disk)
        # For the running example we expect: [[1], [2], [4, 3]]

        # Add state to states
        states.append(pegs)

    return states

Next, we need some code that computes the list of legal actions to take from any given state. This will help us build the actions dictionary.

In [4]:
def get_actions_for_state(state: tuple[tuple[int]]) -> list[str]:
    """Given a state, return all of the valid actions in the form 'i->j'.

    In notation 'i->j', we mean move the top element of peg i to peg j. The leftmost peg is '0'.

    Example: state ((1,), (2,), (4, 3)) has possible actions ['0->1', '0->2', '1->2'].

    Args:
        state (tuple[tuple[int]]): Each sub-tuple represents a peg.
            On each peg are disks represented by integers.
            The order of integers represents the order on the peg from bottom to top.
            Smaller disks are represented by smaller ints (1 is the smallest disk).

    Returns:
        list[str]: Valid actions to take in the state.
    """
    # Store valid actions
    actions = []

    # Get the number of pegs from the state
    num_pegs = len(state)

    # Iterate over the source pegs ('i')
    for i in range(num_pegs):

        # Skip empty pegs with no disks
        if not state[i]:
            continue

        # Get the top disk (should be the smallest on the peg)
        # Pegs are ordered from biggest to smallest so take last element.
        top_disk = state[i][-1]

        # Iterate over the destination pegs ('j')
        for j in range(num_pegs):

            # Skip the action of not moving the disk ('i->i')
            if i == j:
                continue

            # If the destination peg is empty or the current top disk is bigger than the candidate disk - this is a legal action.
            if not state[j] or state[j][-1] > top_disk:
                actions.append(f"{i}->{j}")

    return actions

The next function actually performs the action so that given a state and an action, the next state is found.

In [5]:
def apply_action(state: tuple[tuple[int]], action_str: str) -> tuple[tuple[int]]:
    """Apply an action to a state and return the new state.
    The action is moving the top disk from peg i to peg j.

    Args:
        state (tuple[tuple[int]]): The state representing the pegs and the disks on each peg.
        action_str (str): 'i->j' - move disk from top of source peg i to destination peg j

    Returns:
        tuple[tuple[int]]: The new state after taking the action.
    """
    # Action is in form "i->j"
    # Get source peg and destination peg
    i_str, j_str = action_str.split("->")
    i, j = int(i_str), int(j_str)

    # Convert immutable state to a list to apply the action
    pegs = [list(peg) for peg in state]
    # Remove disk from source peg
    disk = pegs[i].pop()
    # Add disk to end of destimation peg (i.e. top position)
    pegs[j].append(disk)

    # Convert state back to tuples
    return tuple(tuple(peg) for peg in pegs)

Using the above-mentioned helper functions, we can write another function to output the variables we need to model the MDP. This includes:
- **S**: The list of states where a state is a complete configuration of the position of the disks on the pegs.
- **A**: The actions dictionary - the keys of the dictionary are the valid states of the puzzle and the values are the actions which can be taken in each state.
- **P**: The transition probability dictionary - each combination of `(current state, action, next state)` has an associated probability. In this case, because the system is deterministic, all probabilities have value 1.0.
- **R**: The rewards dictionary - each combination of `(current state, action, next state)` has an associated reward. In this problem, all combinations where the next state is the goal state receives a reward. No other combinations of current state, action and next state receive rewards. This means there is a sparse reward structure.

In [6]:
def build_tower_of_hanoi_mdp(
    num_disks: int = 4, num_pegs: int = 3
) -> tuple[list, dict, dict, dict]:
    """Returns S, A, P, R dictionaries for an Tower of Hanoi MDP.

    Args:
        num_disks (int, optional): The number of disks. Defaults to 4.
        num_pegs (int, optional): The number of pegs. Defaults to 3.

    Returns:
        tuple[list, dict, dict, dict]: S, A, P, R variables - the MDP model for the Tower of Hanoi.
    """
    # Generate all states
    states_list = generate_all_states(num_disks, num_pegs)

    # Convert to tuples since the states should not be modified
    S = [tuple(tuple(peg) for peg in state) for state in states_list]

    # Build action dictionary A for all states
    A = {}
    for s_key in S:
        # Per state, list possible actions
        A[s_key] = get_actions_for_state(s_key)

    # Build transition probability matrix P for all states
    P = {}
    for s_key in S:
        for action_str in A[s_key]:
            # Get next state on taking an action
            # The system is deterministic in the sense that when you move a disk, it goes where intended.
            s_prime = apply_action(s_key, action_str)
            P[(s_key, action_str, s_prime)] = 1.0

    # Determine goal state for the rewards dictionary
    # The goal is to move all disks to the last peg in order of biggest to smallest.
    goal_list = [[] for _ in range(num_pegs)]
    # The last peg needs to have entry descending disk sizes
    goal_list[-1] = list(range(num_disks, 0, -1))  # e.g. [4, 3, 2, 1] for 4 disks
    # Expecting a tuple
    goal_state = tuple(tuple(peg) for peg in goal_list)

    # Build rewards dictionary
    # No rewards anywhere except final goal state which has a goal of 1
    R = {}
    for s_key, action_str, s_prime in P:
        if s_prime == goal_state:
            # Goal state reward
            R[(s_key, action_str, s_prime)] = 1
        else:
            R[(s_key, action_str, s_prime)] = 0

    return S, A, P, R

Next we use the above function to build the MDP variables for a problem where there are 4 disks and 3 pegs.

In [7]:
# Build the MDP
S, A, P, R = build_tower_of_hanoi_mdp(num_disks=4, num_pegs=3)

In [8]:
print("Number of States:", len(S))

Number of States: 81


There are 81 states for this problem. 

## Value iteration to find the optimal policy

Next, we compute the optimal policy and the optimal state values using the `ValueIteration` class, which performs synchronous value iteration. 

In [9]:
# Create MDP data class
mdp = MDP(states=S, actions=A, probabilities=P, rewards=R)

# Setup value iteration class
value_itr = ValueIteration(mdp=mdp, gamma=0.95, theta=1e-9, printing=True)

# Run value iteration algorithm
optimal_values, optimal_policy = value_itr.value_iteration()

Iteration 1, max value change: 1.0
Iteration 2, max value change: 0.95
Iteration 3, max value change: 0.9025
Iteration 4, max value change: 0.8573749999999999
Iteration 5, max value change: 0.81450625
Iteration 6, max value change: 0.7737809375
Iteration 7, max value change: 0.7350918906249999
Iteration 8, max value change: 0.69833729609375
Iteration 9, max value change: 0.6634204312890624
Iteration 10, max value change: 0.6302494097246092
Iteration 11, max value change: 0.5987369392383788
Iteration 12, max value change: 0.5688000922764598
Iteration 13, max value change: 0.5403600876626369
Iteration 14, max value change: 0.513342083279505
Iteration 15, max value change: 0.4876749791155297
Iteration 16, max value change: 0.4632912301597534
Iteration 17, max value change: 0.44012666865176553
Iteration 18, max value change: 0.41812033521917735
Iteration 19, max value change: 0.39721431845821886
Iteration 20, max value change: 0.37735360253530814
Iteration 21, max value change: 0.358485922

In [10]:
# Display results
print("Optimal State Values:", optimal_values)
print("Optimal Policy:", optimal_policy)

Optimal State Values: {((4, 3, 2, 1), (), ()): 5.001794648331587, ((3, 2, 1), (4,), ()): 7.162433797289331, ((3, 2, 1), (), (4,)): 7.539403997146664, ((4, 2, 1), (3,), ()): 6.140891675232602, ((2, 1), (4, 3), ()): 5.001794648331587, ((2, 1), (3,), (4,)): 8.793589734789329, ((4, 2, 1), (), (3,)): 5.833847091470972, ((2, 1), (4,), (3,)): 5.001794648331587, ((2, 1), (), (4, 3)): 9.256410247146663, ((4, 3, 1), (2,), ()): 5.265046999194511, ((3, 1), (4, 2), ()): 6.140891675232602, ((3, 1), (2,), (4,)): 8.353910247146665, ((4, 1), (3, 2), ()): 6.804312106521665, ((1,), (4, 3, 2), ()): 5.265046999194511, ((1,), (3, 2), (4,)): 7.539403997146664, ((4, 1), (2,), (3,)): 5.001794648331587, ((1,), (4, 2), (3,)): 5.542154735994222, ((1,), (2,), (4, 3)): 9.74358973478933, ((4, 3, 1), (), (2,)): 5.542154735994222, ((3, 1), (4,), (2,)): 6.140891675232602, ((3, 1), (), (4, 2)): 7.936214734789331, ((4, 1), (3,), (2,)): 6.4640965011955815, ((1,), (4, 3), (2,)): 5.542154735994222, ((1,), (3,), (4, 2)): 7.5

Because there are so many states, we need a better representation of which actions to take to solve the puzzle. In the next sections, we show the optimal actions to take, according to the policy derived using value iteration, from the beginning state of the puzzle to the goal state. This [source](https://en.wikipedia.org/wiki/Tower_of_Hanoi) states that we need a minimum of $2^n -1$ actions to solve the puzzle where $n$ represents the number of disks. The number of disks we are using is 4 so we expect the best solution to have 15 or more steps to get to the goal.

In [11]:
def print_tower_state(state: tuple[tuple[int]]) -> None:
    """Print a Tower of Hanoi state to the terminal in a user-friendly format.

    Args:
        state (tuple[tuple[int]]): Each sub-tuple represents a peg.
            On each peg are disks represented by integers.
            The order of integers represents the order on the peg from bottom to top.
            Smaller disks are represented by smaller ints (1 is the smallest disk).
    """
    # Get the labels for the pegs (i.e. '0', '1',...)
    peg_labels = [str(i) for i in range(len(state))]

    for peg_index, peg_contents in enumerate(state):
        # Peg label
        label = peg_labels[peg_index]

        # Get content of which disks are on each peg
        if peg_contents:
            # Not empty peg (i.e. there are disks)
            disk_str = ", ".join(str(disk) for disk in peg_contents)
        else:
            disk_str = "(empty)"

        # Print peg label and contents
        print(f"  {label}: {disk_str}")


def show_tower_of_hanoi_plan(
    policy: dict[tuple[tuple[int]], str],
    start_state: tuple[tuple[int]],
    goal_state: tuple[tuple[int]],
    max_steps: int = 50,
) -> None:
    """Given a policy, a start state and a goal state, print out each state and action recommended from the start state until either:
        - the goal state is reached
        - a maximum number of steps are taken

    This function:
      1. Prints the current state.
      2. Prints the policy's best action for the current state.
      3. Applies the best action to get the next state.
      4. Repeat until:
        - the goal state is reached
        - OR a maximum number of steps are taken

    Args:
        policy (dict[tuple[tuple[int]], str]): A mapping of states to the best action to take in that state.
        start_state (tuple[tuple[int]]): The beginning state of the puzzle.
        goal_state (tuple[tuple[int]]): The end state of the puzzle.
        max_steps (int, optional): The maximum number of steps to take until stopping. Defaults to 50.
    """
    current_state = start_state
    step = 0

    while True:
        # Increment steps
        step += 1

        # Print the step
        print(f"STEP {step}:")

        # Print current state
        print("Current State:")
        print_tower_state(current_state)

        # Check if the current state is not in the policy
        if current_state not in policy:
            print("No action defined for this state. Stopping.")
            break

        # Get best action for current state
        action = policy[current_state]
        print(f"Best Action: {action}")

        # Apply the action
        next_state = apply_action(current_state, action)

        # Barrier between steps for neatness
        print("\n--------------------------------------------------\n")

        # Check if the goal is reached after the action
        if next_state == goal_state:
            print("Reached the goal state!\n")

            # Print goal state
            print_tower_state(goal_state)

            # End loop
            break

        # Move to next state
        current_state = next_state

        # Check if the maximum number of steps are reached
        if step >= max_steps:
            print(f"Reached {max_steps} steps without getting to goal. Stopping.")
            break

In [12]:
start_state = ((4, 3, 2, 1), (), ())  # All disks on peg 0
goal_state = ((), (), (4, 3, 2, 1))  # All disks on peg 2
show_tower_of_hanoi_plan(
    policy=optimal_policy,
    start_state=start_state,
    goal_state=goal_state,
    max_steps=20,
)

STEP 1:
Current State:
  0: 4, 3, 2, 1
  1: (empty)
  2: (empty)
Best Action: 0->1

--------------------------------------------------

STEP 2:
Current State:
  0: 4, 3, 2
  1: 1
  2: (empty)
Best Action: 0->2

--------------------------------------------------

STEP 3:
Current State:
  0: 4, 3
  1: 1
  2: 2
Best Action: 1->2

--------------------------------------------------

STEP 4:
Current State:
  0: 4, 3
  1: (empty)
  2: 2, 1
Best Action: 0->1

--------------------------------------------------

STEP 5:
Current State:
  0: 4
  1: 3
  2: 2, 1
Best Action: 2->0

--------------------------------------------------

STEP 6:
Current State:
  0: 4, 1
  1: 3
  2: 2
Best Action: 2->1

--------------------------------------------------

STEP 7:
Current State:
  0: 4, 1
  1: 3, 2
  2: (empty)
Best Action: 0->1

--------------------------------------------------

STEP 8:
Current State:
  0: 4
  1: 3, 2, 1
  2: (empty)
Best Action: 0->2

--------------------------------------------------

ST

We reached the goal state by taking only 15 actions. This means that we have the true optimal solution since [source](https://en.wikipedia.org/wiki/Tower_of_Hanoi) stipulates that a minimum of 15 steps are required to solve the puzzle with 4 disks.